# Dataset Analysis: "60,000+ Chess Game Dataset (Chess.com)"

## Author

Analysis by Ivan Lamparelli

Written in October 2021

## Dataset Description

The dataset gathers information regarding chess games on the [Chess.com](https://www.chess.com/) platform. 

The dataset has been retrieved from [Kaggle](https://www.kaggle.com/adityajha1504/chesscom-user-games-60000-games). It's composed of a 160MB .csv file, which contains 14 columns and 66.879 rows.

Each row is related to a single game, and contains data like:
- username and ELO score of both players
- which player has won
- type of victory/draw (i.e. victory by checkmate or time)
- time format
- game format
- game FEN and PGN

## Analysis goals

The initial questions I've planned to respond to when starting this analysis are the following:
- How is ELO distributed among players?
- What are the most common opening moves? 
- How do openings change at different ELO ranges?
- How many turns is an average game?
- How does the time format influence the number of turns and the piece advantage between players?
- Do high ELO players end the game with a lower or higher piece advantage on average?
- What are the most common win/draw conditions overall? Are these influencede by ELO and time format?

Further questions may emerge during the analysis depending on what I discover.

## Project setup

### Project libraries

In [1]:
import pandas as pd
import os

### Dataset import

In [2]:
# I've used the OS path separator to ensure the code would work both on Windows and Unix environments
games = pd.read_csv("data" + os.path.sep + "club_games_data.csv")

## Dataset structure

In [3]:
print("Columns:")
print(list(games.columns))
print("Number of columns: ", games.columns.size)
print("Number of rows: ", len(games))

Columns:
['white_username', 'black_username', 'white_id', 'black_id', 'white_rating', 'black_rating', 'white_result', 'black_result', 'time_class', 'time_control', 'rules', 'rated', 'fen', 'pgn']
Number of columns:  14
Number of rows:  66879


### Sample data

In [4]:
games.head()

,white_username,black_username,white_id,black_id,white_rating,black_rating,white_result,black_result,time_class,time_control,rules,rated,fen,pgn
0,-Amos-,miniman2804,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/miniman2804,1708,1608,win,checkmated,daily,1/259200,chess,True,r2r4/p2p1p1p/b6R/n1p1kp2/2P2P2/3BP3/PP5P/4K2R ...,"[Event ""Enjoyable games 2 - Round 1""]\n[Site ""..."
1,-Amos-,koltcho69,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/koltcho69,1726,1577,win,resigned,daily,1/172800,chess,True,8/5Q1k/4n1pp/8/7P/2N2b2/PP3P2/5K2 b - - 1 33,"[Event ""Rapid Rats - Board 5""]\n[Site ""Chess.c..."
2,-Amos-,enhmandah,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/enhmandah,1727,842,win,resigned,daily,1/172800,chess,True,rn1q1b1r/kb2p1pp/2p5/p1Q5/N1BP2n1/4PN2/1P3PPP/...,"[Event ""CHESS BOARD CLASH - Round 1""]\n[Site ""..."
3,enhmandah,-Amos-,https://api.chess.com/pub/player/enhmandah,https://api.chess.com/pub/player/-amos-,819,1727,checkmated,win,daily,1/172800,chess,True,r3kb1r/pp3ppp/3p1n2/2pKp3/P3P3/1P6/4qP1P/QNB5 ...,"[Event ""CHESS BOARD CLASH - Round 1""]\n[Site ""..."
4,-Amos-,Shalllow-Blue,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/shalllow-blue,1729,1116,win,resigned,daily,1/172800,chess,True,r3b2r/pp6/2pPpR1k/4n3/2P3Q1/3B4/PP4PP/R5K1 b -...,"[Event ""CHESS BOARD CLASH - Round 1""]\n[Site ""..."


The nature of the content in most columns is pretty clear:
- white_username and black_username: players usernames
- white_id and black_id: players unique IDs
- white_rating and black_rating: players ELO scores
- white_result and black_result: contain information regarding the game result
- time_class and time_control: game time format
- rated: whether a game is rated or not
- FEN: FEN notation of the game (indicates the last position of the game)

Two columns require further exploration:
- rules: Each row in the sample data contains "chess" for this column. What does "chess" mean in this context?
- PGN: Why doesn't the sample data show PGN notations?

### Data format of the rules column

In [5]:
# Possible values for the rules column
print(games["rules"].unique())

['chess' 'chess960' 'threecheck' 'crazyhouse' 'kingofthehill']


The rules column indicates whether the game followed traditional chess rules or chess variants

### Data format of the PGN column

In [6]:
# Sample data in the PGN column
print(games.loc[0, "pgn"])

[Event "Enjoyable games 2 - Round 1"]
[Site "Chess.com"]
[Date "2013.01.30"]
[Round "-"]
[White "-Amos-"]
[Black "miniman2804"]
[Result "1-0"]
[Tournament "https://www.chess.com/tournament/enjoyable-games-2"]
[CurrentPosition "r2r4/p2p1p1p/b6R/n1p1kp2/2P2P2/3BP3/PP5P/4K2R b K f3 1 22"]
[Timezone "UTC"]
[ECO "E22"]
[ECOUrl "https://www.chess.com/openings/Nimzo-Indian-Defense-Spielmann-Variation"]
[UTCDate "2013.01.30"]
[UTCTime "16:35:14"]
[WhiteElo "1708"]
[BlackElo "1608"]
[TimeControl "1/259200"]
[Termination "-Amos- won by checkmate"]
[StartTime "16:35:14"]
[EndDate "2013.02.01"]
[EndTime "18:14:48"]
[Link "https://www.chess.com/game/daily/64629816"]

1. d4 Nf6 2. c4 e6 3. Nc3 Bb4 4. Qb3 Bxc3+ 5. Qxc3 O-O 6. Bg5 c5 7. dxc5 Nc6 8. Nf3 Qa5 9. Bxf6 gxf6 10. Qxa5 Nxa5 11. e3 Rd8 12. Rd1 Kg7 13. Be2 b6 14. Rd4 bxc5 15. Rg4+ Kh6 16. Bd3 f5 17. Rh4+ Kg6 18. g4 Ba6 19. gxf5+ exf5 20. Ne5+ Kf6 21. Rh6+ Kxe5 22. f4# 1-0



The PGN column contains nested data. During the data manipulation phase, we'll extract these data.

## Missing values

### Number of NA values

We can check the number of NA values per column by using `isna().sum()`.

Then we can count the total number of NA values in the dataset by summing the NA values in each group.

In [7]:
print("NA values: ", games.isna().sum().sum())

NA values:  0


There are no NA values in the dataset.

### Number of empty strings

To check for missing values expressed as empty strings, first we must manipulate the dataset by trimmming initial and ending spaces in string values.

We can achieve this by the strip() function to each column that contains strings.

In [8]:
games = games.applymap(lambda colData: colData.strip() if isinstance(colData, str) else colData)

We can then transform empty strings into NA values.

In [9]:
games = games.replace("", pd.NA)

Now we check again for NA values.

In [10]:
print("Empty strings: ", games.isna().sum().sum())

Empty strings:  0


There are no empty strings in the dataset.

## Initial manipulation

After an initial exploration of the data structure, I've set an initial list of manipulations I want to perform on the raw data to prepare for the analysis:
- Determine if we can drop the player id columns
- Move the game win/draw condition to a single column (it's currently split between the columns white_result and black_result)
- Extract piece advantage and point advantage at end of game from the FEN column
- Extract date, number of turns, PGN notation from the PGN column

### Players manipulation

#### Checking username uniqueness

We want to determine whether we can remove the winner_id and loser_id columns.

In order to do so, we must check whether there are players with different IDs that have the same username.

In [11]:
# Extracts username-id pairs for white and black players
whitePlayers = games[["white_username", "white_id"]].rename(columns={"white_username": "username", "white_id": "id"})
blackPlayers = games[["black_username", "black_id"]].rename(columns={"black_username": "username", "black_id": "id"})

# Gathers the username-id data of every game in a common dataframe
players = pd.concat([whitePlayers, blackPlayers])

# Gathers the unique username-id pairs for each player
players = players.drop_duplicates()

# Prints the number of duplicated usernames
numOfPlayerByUsername = players["username"].value_counts()
print("Duplicated usernames: ", numOfPlayerByUsername[numOfPlayerByUsername > 1].size)

Duplicated usernames:  0


#### Removing redundant data

After having determined that usernames are valid unique identifiers, we can drop the winner_id and loser_id columns.

In [12]:
games.drop(columns=["white_id", "black_id"], inplace=True)
print(list(games.columns))

['white_username', 'black_username', 'white_rating', 'black_rating', 'white_result', 'black_result', 'time_class', 'time_control', 'rules', 'rated', 'fen', 'pgn']


### Game results manipulation

Currently, the win/draw conditions are divided between two columns (white_result and black_result).

For drawn games, both columns contain the same information, that explains the type of draw (i.e. stalemate, timeout vs insufficient material, etc).
For games with a winner, the data contains the word "win" on the column related to the player who won, and the type of defeat on the column related to the other player.

This structure is unfit for exploring the different types of win/draw conditions. We'll proceed to split these information in two columns:
- winner (which can be: white, black, neither)
- result type (the win/draw cause; i.e. checkmate, timeout)

#### Extracting winning player

In [14]:
# Defined constants to avoid using hardwired strings in code
winnerInCaseOfDraw = "neither"
winResult = "win"
whitePlayer = "white"
blackPlayer = "black"

# Sets a unique column with the information on who won
games["winner"] = winnerInCaseOfDraw
games.loc[games["white_result"] == winResult, "winner"] = whitePlayer
games.loc[games["black_result"] == winResult, "winner"] = blackPlayer

print("Sample mapping of the winner column:")
print(games[["white_result", "black_result", "winner"]].head())

Sample mapping of the winner column:
  white_result black_result winner
0          win   checkmated  white
1          win     resigned  white
2          win     resigned  white
3   checkmated          win  black
4          win     resigned  white


#### Extracting game result type

In [15]:
# Defines how to read the result type from each game
def getResultType(game):
    return game["black_result"] if game["winner"] == whitePlayer else game["white_result"]

# Sets the result type in a new column
games["result_type"] = games.apply(lambda game: getResultType(game), axis=1)

print("Sample mapping of the new columns:")
print(games[["white_result", "black_result", "winner", "result_type"]].head())

Sample mapping of the new columns:
  white_result black_result winner result_type
0          win   checkmated  white  checkmated
1          win     resigned  white    resigned
2          win     resigned  white    resigned
3   checkmated          win  black  checkmated
4          win     resigned  white    resigned


#### Dropping old columns

In [16]:
games.drop(columns=["white_result", "black_result"], inplace=True)
print("Columns: ", list(games.columns))

Columns:  ['white_username', 'black_username', 'white_rating', 'black_rating', 'time_class', 'time_control', 'rules', 'rated', 'fen', 'pgn', 'winner', 'result_type']


### FEN data manipulation

#### Hints about FEN notation

A FEN string contains a series of information regarding a particular state of the board:
- which pieces are present in each row of the board
    - each row is separated by a slash
    - each row is represented as a series of letters and numbers (a letter is a piece, a number is a set of empty spaces)
    - K is King, Q is Queen, R is rook, N is knight, B is bishop, P is pawn
    - white pieces are in uppercase, black pieces are in lowercase
- which player has to move
- castling options
- en passant options
- number of turns

The main interest of the analysis regarding this data is in the piece and points advantage, which we'll extract first. A possible future development is extracting and analyzing information like castling and en passant options for each game.

#### Hints about piece and points advantage

Each player start with 16 pieces. Each piece has a different score, which is used to evaluate which player has an advantage in material:
- king: no score (since it can't be captured, it can only be checkmated)
- queen: 9 points
- rook: 5 points
- bishop: 3 points
- knight: 3 points
- pawn: 1 point

Sometimes two players have the same score in terms of remaining pieces, but one of them has a higher number of pieces (pieces with a minor score). This can mean occupying more phisical squares on the board, which, according to chess litetature, can be an important factor during the endgame (the ending phase of the game).

Part of the analysis goals is understanding how differences in terms of available pieces and point advantage reflect on winrates.

#### Extracting board setup from FEN notation

First, we'll extract the information about the board setup from the FEN, to allow us to count the different pieces without considering the additional information in a FEN string.

Each group of information in a valid FEN string is separated by a space. The board setup counts as a single piece of information, and it's written as a series of 8 strings separated by slashes. 

We could simply extract the substring up to the first space, but this method would be prone to errors, in case of invalid FEN strings. By using a regular expression to check for substrings with a valid format, we can ensure that the data is valid.

In [17]:
# A single row can contain 1 to 8 values, letters or number
# A letter indicates a pieces
# A number indicates the empty spaces to the right (so, minimum 1 space to be specified, max 8 spaces being the 8 cells of a chessboard row) 
validCharsForOneRow = "[rnbqkpRNBQKP1-8]{1,8}"

# There are 8 rows. Each row (except the last one) has a slash to the right
# We're looking for data at the start of the string, so we use the ^ symbol
boardRegex = "^((" + validCharsForOneRow + "/){7}" + validCharsForOneRow + ")"

# There are 2 capture groups (parts nested in parenthesis): First is the whole row. Then the first 7 rows (needed to repeat the check 7 times).
# We need the first capture groups, that represents the whole board
games["fen_board_setup"] = games["fen"].str.extract(boardRegex)[0]

print("Number of games with invalid board information: ", games["fen_board_setup"].isna().sum())
print("Sample board setups:")
print(games["fen_board_setup"].head(3))

Number of games with invalid board information:  0
Sample board setups:
0        r2r4/p2p1p1p/b6R/n1p1kp2/2P2P2/3BP3/PP5P/4K2R
1                    8/5Q1k/4n1pp/8/7P/2N2b2/PP3P2/5K2
2    rn1q1b1r/kb2p1pp/2p5/p1Q5/N1BP2n1/4PN2/1P3PPP/...
Name: fen_board_setup, dtype: object


#### Number of pieces

After having extracted the board setup, we can count how many pieces each player has remaining at the end of the game.

In [18]:
# Counts white pieces
games["white_pieces_kings"] = games["fen_board_setup"].str.count("K")
games["white_pieces_queens"] = games["fen_board_setup"].str.count("Q")
games["white_pieces_rooks"] = games["fen_board_setup"].str.count("R")
games["white_pieces_knights"] = games["fen_board_setup"].str.count("N")
games["white_pieces_bishops"] = games["fen_board_setup"].str.count("B")
games["white_pieces_pawns"] = games["fen_board_setup"].str.count("P")

# Counts black pieces
games["black_pieces_kings"] = games["fen_board_setup"].str.count("k")
games["black_pieces_queens"] = games["fen_board_setup"].str.count("q")
games["black_pieces_rooks"] = games["fen_board_setup"].str.count("r")
games["black_pieces_knights"] = games["fen_board_setup"].str.count("n")
games["black_pieces_bishops"] = games["fen_board_setup"].str.count("b")
games["black_pieces_pawns"] = games["fen_board_setup"].str.count("p")

# Counts totals
games["white_pieces"] = games.loc[:, "white_pieces_kings":"white_pieces_pawns"].sum(axis=1)
games["black_pieces"] = games.loc[:, "black_pieces_kings":"black_pieces_pawns"].sum(axis=1)

# Sample data
sampleGame = games.iloc[0, :]
print("Sample comparison between FEN board and number of pieces:")
print(sampleGame["fen_board_setup"])
print("White pieces: ", sampleGame["white_pieces"])
print("White knights: ", sampleGame["white_pieces_knights"])
print("Black pieces: ", sampleGame["black_pieces"])
print("Black knights: ", sampleGame["black_pieces_knights"])

Sample comparison between FEN board and number of pieces:
r2r4/p2p1p1p/b6R/n1p1kp2/2P2P2/3BP3/PP5P/4K2R
White pieces:  10
White knights:  0
Black pieces:  11
Black knights:  1


#### Piece advantage

Now we'll calculate the difference in number of pieces between the winner and the loser. 
If the number is negative, it means that the winner has less pieces remaining than the loser.

In [19]:
# Can be used to calculate points or pieces advantage by passing the desired column name
def getAdvantage(game, columnName):
    return (game["white_" + columnName] - game["black_" + columnName]) \
        * (1 if game["winner"] == whitePlayer else -1) # If White has won, we calculate normally; otherwise, we flip the result

games["piece_advantage"] = games.apply(lambda game: getAdvantage(game, "pieces"), axis=1)

# Sample data
print("Sample piece advantage data:")
print(games[["winner", "white_pieces", "black_pieces", "piece_advantage"]].head(3))

Sample piece advantage data:
  winner  white_pieces  black_pieces  piece_advantage
0  white            10            11               -1
1  white             7             5                2
2  white            14            13                1


#### Points advantage

Now we'll calculate the difference in pieces points between the winner and the loser.
If the number is negative, it means that the winner has pieces with less total points remaining than the loser.

In [20]:
def getPoints(game, player):
    return game[player + "_pieces_queens"] * 9 + game[player + "_pieces_rooks"] * 5 + \
    game[player + "_pieces_knights"] * 3 + game[player + "_pieces_bishops"] * 3 + game[player + "_pieces_pawns"] * 1

games["white_points"] = games.apply(lambda game: getPoints(game, whitePlayer), axis=1)
games["black_points"] = games.apply(lambda game: getPoints(game, blackPlayer), axis=1)
games["points_advantage"] = games.apply(lambda game: getAdvantage(game, "points"), axis=1)

# Sample data
sampleGame = games.iloc[0, :]
print ("Sample point calculation: ")
print("White points: ", sampleGame["white_points"])
print("White queens (9pt each): ", sampleGame["white_pieces_queens"])
print("White rooks (5pt each): ", sampleGame["white_pieces_rooks"])
print("White knights (3pt each): ", sampleGame["white_pieces_knights"])
print("White bishops (3pt each): ", sampleGame["white_pieces_bishops"])
print("White pawns (1pt each): ", sampleGame["white_pieces_pawns"])

print("\nSample points advantage data:")
print(games[["winner", "white_points", "black_points", "points_advantage"]].head(3))

Sample point advantage calculation: 
White points:  19
White queens (9pt each):  0
White rooks (5pt each):  2
White knights (3pt each):  0
White bishops (3pt each):  1
White pawns (1pt each):  6

Sample points advantage data:
  winner  white_points  black_points  points_advantage
0  white            19            22                -3
1  white            16             8                 8
2  white            37            36                 1


### PGN column manipulation

As seen in the initial data exploration, the PGN column contains nested data.

In [21]:
print(games.loc[0, "pgn"])

[Event "Enjoyable games 2 - Round 1"]
[Site "Chess.com"]
[Date "2013.01.30"]
[Round "-"]
[White "-Amos-"]
[Black "miniman2804"]
[Result "1-0"]
[Tournament "https://www.chess.com/tournament/enjoyable-games-2"]
[CurrentPosition "r2r4/p2p1p1p/b6R/n1p1kp2/2P2P2/3BP3/PP5P/4K2R b K f3 1 22"]
[Timezone "UTC"]
[ECO "E22"]
[ECOUrl "https://www.chess.com/openings/Nimzo-Indian-Defense-Spielmann-Variation"]
[UTCDate "2013.01.30"]
[UTCTime "16:35:14"]
[WhiteElo "1708"]
[BlackElo "1608"]
[TimeControl "1/259200"]
[Termination "-Amos- won by checkmate"]
[StartTime "16:35:14"]
[EndDate "2013.02.01"]
[EndTime "18:14:48"]
[Link "https://www.chess.com/game/daily/64629816"]

1. d4 Nf6 2. c4 e6 3. Nc3 Bb4 4. Qb3 Bxc3+ 5. Qxc3 O-O 6. Bg5 c5 7. dxc5 Nc6 8. Nf3 Qa5 9. Bxf6 gxf6 10. Qxa5 Nxa5 11. e3 Rd8 12. Rd1 Kg7 13. Be2 b6 14. Rd4 bxc5 15. Rg4+ Kh6 16. Bd3 f5 17. Rh4+ Kg6 18. g4 Ba6 19. gxf5+ exf5 20. Ne5+ Kf6 21. Rh6+ Kxe5 22. f4# 1-0


Some of the data in it (i.e. white/black ELO, termination cause, etc.) are redundant with other columns in the dataset. Other pieces of data are less interesting from an analysis standpoint (i.e. timezone, url link to game information).

The two pieces of data we're interested in extracting are:
- game date (to perform analysis on dates)
- game PGN (to perform analysis on moves)

First thing we'll do is renaming the PGN column into a less ambiguous name (so that we can then use the `pgn` name for the column that will contain only pgn data).
We won't remove the current PGN column since we may be interested in extracting additional data from it, depending on the result of the analysis.

In [22]:
games.rename(columns={"pgn": "game_information"}, inplace=True)

#### Extracting game date from PGN column

We'll use a regular expression to extract the game date from the game_information column, looking for the values contained within the [Date] tag.

In [23]:
# A date in the game_information column is composed as year.month.day
dateStringRegex = "[0-9]{4}\.[0-9]{2}\.[0-9]{2}"
# The date is contained in a tag in the structure [Date "my_date"]
# We only want the my_date part, so we put it in a capture group (in parenthesis)
dateFieldRegex = "\[Date \"(" + dateStringRegex + ")\"\]"

games["game_date"] = games["game_information"].str.extract(dateFieldRegex)[0]

# Sample data
print("Number of games with invalid date information: ", games["game_date"].isna().sum())
print("Sample dates:")
print(games["game_date"].head(3))

Number of games with invalid date information:  0
Sample dates:
0    2013.01.30
1    2013.01.19
2    2013.02.01
Name: game_date, dtype: object


#### Extracting PGN moves data

A valid PGN string is composed as a series of numbers (representing turns), with white and black moves for each turn.

Example: `1. d4 Nf6 2. c4 e6 3. Nc3 Bb4 [...]`